In [9]:
import warnings
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import (
                            StratifiedKFold,
                            GridSearchCV
                            )
from sklearn.metrics import confusion_matrix
warnings.filterwarnings('ignore')

### 01. Load the data


In [10]:
X_train = np.load('artifacts/X_train.npz')['arr_0']
Y_train = np.load('artifacts/Y_train.npz')['arr_0']
X_test = np.load('artifacts/X_test.npz')['arr_0']
Y_test = np.load('artifacts/Y_test.npz')['arr_0']

### 02. Hyper Param Tuning

In [11]:
lr_param_grid = {
                'max_iter' : [1000, 5000, 10000]                 
                }# 3 different LR models
dt_param_grid = {
                'max_depth' : [8, 12, 16, 20],
                'criterion' : ["gini", "entropy", "log_loss"]              
                }# 12 (4 * 3) different DT models
rf_param_grid = {
                'n_estimators' : [50, 100, 150, 200],
                'max_depth' : [8, 12, 16, 20],
                'criterion' : ["gini", "entropy", "log_loss"]              
                }# 48 (4 * 4 * 3) different RF models

param_grids = {
        'Logistic Regression' : lr_param_grid,
        'Decision Tree' : dt_param_grid,
        'Random Forest' : rf_param_grid,
        }

#### 03. Define Multi Models

In [12]:
models = {
        'Logistic Regression' : LogisticRegression(),
        'Decision Tree' :DecisionTreeClassifier(),
        'Random Forest' : RandomForestClassifier()
        }

### 04. Configure K-Fold CV

In [13]:
cv = StratifiedKFold(
                    n_splits=6,
                    random_state=42,
                    shuffle=True
                    )

#### 05. Multi Model Training

In [14]:
grid_search_results = {}
for model_name, model in models.items():
    print(f"\n--- Tuning {model_name} ---")

    param_grid = param_grids[model_name]

    grid_search = GridSearchCV(
                                estimator=model,
                                param_grid=param_grid,
                                cv=cv, scoring='f1',
                                verbose=1, return_train_score=False
                                )
    
    print(f"Fitting gridSearchCV for {model_name}")

    grid_search.fit(X_train, Y_train)

    grid_search_results[model_name] = grid_search
    
    print(f"{model_name} gridSearchCV completed ...")
    print(f"Best parameters: {grid_search.best_params_}")
    print(f"Best CV score: {grid_search.best_score_}")



--- Tuning Logistic Regression ---
Fitting gridSearchCV for Logistic Regression
Fitting 6 folds for each of 3 candidates, totalling 18 fits
Logistic Regression gridSearchCV completed ...
Best parameters: {'max_iter': 1000}
Best CV score: 0.7155921387324043

--- Tuning Decision Tree ---
Fitting gridSearchCV for Decision Tree
Fitting 6 folds for each of 12 candidates, totalling 72 fits
Decision Tree gridSearchCV completed ...
Best parameters: {'criterion': 'entropy', 'max_depth': 20}
Best CV score: 0.838625208555992

--- Tuning Random Forest ---
Fitting gridSearchCV for Random Forest
Fitting 6 folds for each of 48 candidates, totalling 288 fits
Random Forest gridSearchCV completed ...
Best parameters: {'criterion': 'entropy', 'max_depth': 20, 'n_estimators': 200}
Best CV score: 0.8988553009691499


In [15]:
grid_search_results

{'Logistic Regression': GridSearchCV(cv=StratifiedKFold(n_splits=6, random_state=42, shuffle=True),
              estimator=LogisticRegression(),
              param_grid={'max_iter': [1000, 5000, 10000]}, scoring='f1',
              verbose=1),
 'Decision Tree': GridSearchCV(cv=StratifiedKFold(n_splits=6, random_state=42, shuffle=True),
              estimator=DecisionTreeClassifier(),
              param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                          'max_depth': [8, 12, 16, 20]},
              scoring='f1', verbose=1),
 'Random Forest': GridSearchCV(cv=StratifiedKFold(n_splits=6, random_state=42, shuffle=True),
              estimator=RandomForestClassifier(),
              param_grid={'criterion': ['gini', 'entropy', 'log_loss'],
                          'max_depth': [8, 12, 16, 20],
                          'n_estimators': [50, 100, 150, 200]},
              scoring='f1', verbose=1)}